## Production of dredging vessels

### 1. Production of Trailing Suction Hopper Dredger
Production rate of trailing suction hopper dredgers (TSHDs) is a metric for calculating the amount of dredged sediments during a single dredging cycle. The more the production rate of TSHDs, the less time will be spent for conducting the project. Based on the sailing distance to discharging area, capacity of hopper, operational hours (excluding maintenance hours), and sediment types (mud, sand, clay, gravel, etc.), the production of TSHDs varies over the time. Consequently, the whole project can be done with relatively lower corresponding costs. Different determining factors can affect the production of TSHDs summarized as follows.
	Propulsion power (for both sailing and dredging operations)
	Pumping power (filling the hopper and pumping the sediments ashore)
	Erosion power (jet power installed on both drag head and hopper)
	Vacuum (suction power to elevate the slurry)
	Manpower (efficiency of crew and the level of their proficiency)
	Hopper capacity
The total production cycle of TSHDs is calculated using the following equation (Bray et al., 1997).

P=  (C_H f_e)/B(t_load+t_turn+t_sail+t_discharge ) 	(1)
 
where P is the production rate in m^3⁄h, C_H is the hopper capacity in m^3, f_e is the proportion of the hopper that is filled, B is the bulking factor which shows how much of the hopper capacity will be filled with materials during the loading phase and is determined as (dredged volume)/(in-situ volume), and (t_load+t_turn+t_sail+t_discharge ) is different components of a single dredging cycle in hours. A more elaborated calculation of each items in the abovementioned equation is proposed as follows. Due to simplicity, the proportion of the hopper that is filled f_e is considered equal to the ratio of solids to the amount of slurry C_v calculated as follows.
 
f_e=C_v=(〖SG〗_s-〖SG〗_f)/(〖SG〗_so-〖SG〗_f )	(2)

where 〖SG〗_s, 〖SG〗_f, and 〖SG〗_so are specific gravity of slurry, carrier fluid, and solids. Turning time t_turn in hours can be calculated as follows.

t_turn=(N_turn t_load t_180)/L	(3)

where N_turn, t_180, and L are number of turns, the time it takes to turn the dredger around 180 degrees in hours, and the length of the dredging area in nautical miles respectively. Sailing time t_sail which is the time spent to travel from dredging site to discharging site and sail back to the dredging site is calculated as follows.

t_sail=2Y/V_f 	(4)

where Y is the distance between dredging and discharging sites in nautical miles and V_f is the speed of TSHD in knots when the hopper is fully loaded. Discharging time t_discharge is different based on the discharging strategy (bottom-dumped, pumped to shore, or rainbowing). The loading time of hopper t_load depends on using overflow. When overflowing is not allowed, the loading time is calculated as follows.

t_load=C_H/Q	(no overflow)	(5)

where Q is the optimal rate of flow in the hopper in m^3⁄h. When the overflow is allowed, the efficiency of loading the hopper is increased and the production will be maximized until a high concentration of solids start to discharge from top of the hopper. Then, the loading time is increased by the duration of overflowing and it can be calculated as follows.

t_load=C_H/Q+t_overflow	(overflow)	(6)

The production rate shown in Equation (1) is calculated based on an ideal efficiency of operations; while some risk factors can be considered to decrease the production rate of TSHDs. Some of these risk factors that lead to decreasing the production rate are delay factor n_d due to maritime traffic and bad weather conditions, operational factor n_o due to inefficiency of crew, and mechanical breakdown factor n_b due to inevitable breakdown of equipment. An average production rate which is more realistic for real-world problems can be calculated as follows.

P_ave=n_d×n_o×n_b×P	(7)
